# Faiss (Async)

>[Facebook AI Similarity Search (Faiss)](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) 是一个用于高效相似性搜索和密集向量聚类的库。它包含在任何大小的向量集中进行搜索的算法，直至那些可能不适合放入 RAM 的向量。此外，它还包括用于评估和参数调优的配套代码。
>
>请参阅 [The FAISS Library](https://arxiv.org/pdf/2401.08281) 论文。

[Faiss 文档](https://faiss.ai/)。

您需要安装 `langchain-community` (`pip install -qU langchain-community`) 才能使用此集成

本 Notebook 展示了如何使用与 `FAISS` 向量数据库相关的功能，并利用 `asyncio`。
LangChain 已实现同步和异步向量存储函数。

请通过[此处](/docs/integrations/vectorstores/faiss)查看`同步`版本。

In [ ]:
%pip install --upgrade --quiet  faiss-gpu # For CUDA 7.5+ Supported GPU's.
# OR
%pip install --upgrade --quiet  faiss-cpu # For CPU Installation

我们要使用 OpenAIEmbeddings，所以我们必须获取 OpenAI API 密钥。

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Uncomment the following line if you need to initialize FAISS with no AVX2 optimization
# os.environ['FAISS_NO_AVX2'] = '1'

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../../extras/modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

db = await FAISS.afrom_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = await db.asimilarity_search(query)

print(docs[0].page_content)

## 相似度搜索及评分

有些 FAISS 特有的方法。其中一个就是 `similarity_search_with_score`，它允许你不仅返回文档，还能返回查询与这些文档的距离评分。返回的距离评分是 L2 距离。因此，评分越低越好。

In [ ]:
docs_and_scores = await db.asimilarity_search_with_score(query)

docs_and_scores[0]

也可以使用 `similarity_search_by_vector` 对给定嵌入向量的文档进行搜索，该方法接受一个嵌入向量作为参数，而不是字符串。

In [10]:
embedding_vector = await embeddings.aembed_query(query)
docs_and_scores = await db.asimilarity_search_by_vector(embedding_vector)

## 保存和加载
你也可以保存和加载 FAISS 索引。这样做的好处是你不需要每次使用它时都重新创建它。

In [ ]:
db.save_local("faiss_index")

new_db = FAISS.load_local("faiss_index", embeddings, asynchronous=True)

docs = await new_db.asimilarity_search(query)

docs[0]

# 序列化和反序列化为字节

您可以通过以下函数来 pickle FAISS 索引。如果您使用的是 90MB 的 embeddings 模型（如 sentence-transformers/all-MiniLM-L6-v2 或其他任何模型），则生成的 pickle 文件大小将超过 90MB。模型的大小也包含在总体大小中。为了克服这个问题，请使用下面的函数。这些函数仅序列化 FAISS 索引，大小会小得多。如果您希望将索引存储在像 sql 这样的数据库中，这将非常有用。

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

pkl = db.serialize_to_bytes()  # serializes the faiss index
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.deserialize_from_bytes(
    embeddings=embeddings, serialized=pkl, asynchronous=True
)  # Load the index

## 合并
你也可以合并两个 FAISS vectorstores

In [19]:
db1 = await FAISS.afrom_texts(["foo"], embeddings)
db2 = await FAISS.afrom_texts(["bar"], embeddings)

In [20]:
db1.docstore._dict

{'8164a453-9643-4959-87f7-9ba79f9e8fb0': Document(page_content='foo')}

In [21]:
db2.docstore._dict

{'4fbcf8a2-e80f-4f65-9308-2f4cb27cb6e7': Document(page_content='bar')}

In [22]:
db1.merge_from(db2)

In [23]:
db1.docstore._dict

{'8164a453-9643-4959-87f7-9ba79f9e8fb0': Document(page_content='foo'),
 '4fbcf8a2-e80f-4f65-9308-2f4cb27cb6e7': Document(page_content='bar')}

## 带过滤的相似性搜索
FAISS 向量库也支持过滤，由于 FAISS 本身不支持过滤，所以需要手动实现。这可以通过首先获取比 `k` 更多的结果，然后再进行过滤来实现。你可以根据元数据来过滤文档。调用任何搜索方法时，你还可以设置 `fetch_k` 参数来指定在过滤之前想获取多少个文档。这里有一个简单的例子：

In [ ]:
from langchain_core.documents import Document

list_of_documents = [
    Document(page_content="foo", metadata=dict(page=1)),
    Document(page_content="bar", metadata=dict(page=1)),
    Document(page_content="foo", metadata=dict(page=2)),
    Document(page_content="barbar", metadata=dict(page=2)),
    Document(page_content="foo", metadata=dict(page=3)),
    Document(page_content="bar burr", metadata=dict(page=3)),
    Document(page_content="foo", metadata=dict(page=4)),
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
db = FAISS.from_documents(list_of_documents, embeddings)
results_with_scores = db.similarity_search_with_score("foo")
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 5.159960813797904e-15
Content: foo, Metadata: {'page': 2}, Score: 5.159960813797904e-15
Content: foo, Metadata: {'page': 3}, Score: 5.159960813797904e-15
Content: foo, Metadata: {'page': 4}, Score: 5.159960813797904e-15


现在我们发出相同的查询调用，但将筛选条件设为仅 `page = 1`

In [26]:
results_with_scores = await db.asimilarity_search_with_score("foo", filter=dict(page=1))
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 5.159960813797904e-15
Content: bar, Metadata: {'page': 1}, Score: 0.3131446838378906


同样的事情也可以用 `max_marginal_relevance_search` 来完成。

In [27]:
results = await db.amax_marginal_relevance_search("foo", filter=dict(page=1))
for doc in results:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: foo, Metadata: {'page': 1}
Content: bar, Metadata: {'page': 1}


以下是调用 `similarity_search` 时如何设置 `fetch_k` 参数的示例。通常情况下，你希望 `fetch_k` 参数 >> `k` 参数。这是因为 `fetch_k` 参数是指在过滤之前将要获取的文档数量。如果你将 `fetch_k` 设置为一个较低的数字，你可能无法获得足够的文档来进行过滤。

In [4]:
results = await db.asimilarity_search("foo", filter=dict(page=1), k=1, fetch_k=4)
for doc in results:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: foo, Metadata: {'page': 1}


一些 [MongoDB 查询和投影运算符](https://www.mongodb.com/docs/manual/reference/operator/query/) 支持更高级的元数据过滤。当前支持的运算符列表如下：
- `$eq` (等于)
- `$neq` (不等于)
- `$gt` (大于)
- `$lt` (小于)
- `$gte` (大于或等于)
- `$lte` (小于或等于)
- `$in` (在列表中)
- `$nin` (不在列表中)
- `$and` (所有条件必须匹配)
- `$or` (任何条件必须匹配)
- `$not` (条件否定)

使用上面相同的相似性搜索并结合高级元数据过滤，可以执行如下操作：

In [ ]:
results = await db.asimilarity_search(
    "foo", filter={"page": {"$eq": 1}}, k=1, fetch_k=4
)
for doc in results:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: foo, Metadata: {'page': 1}


## 删除

你也可以删除 id。注意要删除的 id 应该是 docstore 中的 id。

In [4]:
db.delete([db.index_to_docstore_id[0]])

True

In [6]:
# Is now missing
0 in db.index_to_docstore_id

False